This script takes Texas Education Agency data about school district demographics and disciplinary actions, and puts them together in one CSV file for the Texas Appleseed "School to Prison Pipeline" map. See http://www.texasdisciplinelab.org/

To use the script, follow these instructions:

1. For every year that you want to cover, download all 20 of the region files from http://rptsvr1.tea.texas.gov/adhocrpt/Disciplinary_Data_Products/Download_Region_Districts.html and put them in the directory '../data/from_agency/by_region/'

2. For every year that you want to cover, download the "District and Charter Detail Data" Snapshot Data File (comma-delimited *.dat)" from https://rptsvr1.tea.texas.gov/perfreport/snapshot/download.html. The website automatically delivers these files with the same filename: district.dat. You will need to rename them to have different names by adding the year after "district". For instance, "district2016.dat"

3. Change the first_year and last_year variables below to reflect the years you want your file to cover.

4. Run the notebook with "Kernel -> Restart and Run All"

5. Wait a while for it to finish. After about 5 minutes, the notebook should produce 'schoolToPrison_flat.csv' in the '../data/processed/' directory.

The map needs a (GeoJSON or TopoJSON) file of district shapes, and it needs a way to select data from the CSV and populate the map with it based on a user's input. This notebook doesn't solve that problem; it only makes the CSV. The resulting file will be about 5.2 MB for 2006-2016, or more depending on how many years it covers. The GeoJSON or TopoJSON file will take additional memory.

The output CSV will have 2 index rows, for the school district's ID number and the year. Then there are 90 more rows as follows:

for each of the 10 demographic groups ('SPE', 'ECO','HIS','BLA', 'WHI','IND', 'ASI','PCI', 'TWO', 'ALL'):
   
    there's one 'POP' column showing the population of that group within the district
   
    there are four punishment columns ('EXP','DAE','ISS','OSS') showing how many times a member of that group received the specified punishment in the district in one year
   
    there are four scale columns ('EXP-S','DAE-S','ISS-S','OSS-S') showing what color the district should be filled with on the map, to represent whether the group's treatment was statistically significant in favor of the group or against. The scale is from -6 to 6, with negative numbers representing fewer than average punishments and positive numbers representing more than average.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats

pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

first_year = 2006 # the year 2006 is the first year on the TEA site
last_year = 2016

pdict = {"EXPULSIONS":"EXP","DAEP REMOVALS":"DAE","IN SCHOOL SUSPENSIONS":"ISS","OUT OF SCHOOL SUSPENSIONS":"OSS"}

demos = {'SPE', 'ECO','HIS','BLA', 'WHI','IND', 'ASI','PCI', 'TWO', 'ALL'}

In [2]:
def formatDF(apple, year, year_col):
    
    # Removes columns not needed for the map
    
    apple = apple.drop(["AGGREGATION LEVEL","REGION","DISTNAME"], axis = 1)
    
    # Keeping only the rows that categorize students by protected class, or that have totals.
    
    patternIn = "WHITE|AFRICAN AMERICAN|AMERICAN INDIAN OR ALASKA NAT|HISPANIC|NATIVE HAWAIIAN|ASIAN" +\
                "|TWO OR MORE RACES|SPEC. ED|ECO. DISAD|ECO DISAD.|TOTAL|DISTRICT CUMULATIVE YEAR END ENROLLMENT" +\
                "|MANDATORY|DISCRETIONARY|NATIVE AMERICAN"
    
    apple = apple[apple["HEADING NAME"].str.contains(patternIn)]
    
    # Getting rid of rows that count students instead of incidents, or non-disadvantaged kids.
    
    patternOut = "SPEC. ED. STUDENTS|EXPULSIONS TO JJAEP|ECO DISAD. STUDENTS|ECO. DISAD. STUDENTS" +\
                 "|AT RISK|NON AT|UNKNOWN AT|NON ECO DISAD.|NON ECO. DISAD."
    apple = apple[apple["HEADING NAME"].str.contains(patternOut) == False]

    # Delete rows appearing to double-count the same expulsions.
    
    JJAEPReplace = {"SECTION": {
                        'M-ECO\. DISADV\. JJAEP PLACEMENTS|H-SPEC\. ED\. JJAEP EXPULSIONS': 'C-JJAEP EXPULSIONS'}}
    apple = apple.replace(to_replace=JJAEPReplace, regex=True)
    apple = apple[apple["SECTION"].str.contains("JJAEP EXPULSIONS|DISCIPLINE ACTION COUNTS") == False]
    
    # Consolidating some of the descriptors into broader categories
    
    appleReplace = {year_col:
                        {-99999999: 1, -999999: 1, -999: 1},
                    "SECTION": {
                        'A-PARTICIPATION': 'POP',
                        'D-EXPULSION ACTIONS|N-ECO\. DISADV\. EXPULSIONS|I-SPEC\. ED\. EXPULSIONS': 'EXP',
                        'E-DAEP PLACEMENTS|O-ECO\. DISADV\. DAEP PLACEMENTS|J-SPEC\. ED\. DAEP PLACEMENTS': 'DAE',
                        'F-OUT OF SCHOOL SUSPENSIONS|P-ECO\. DISADV\. OUT OF SCHOOL SUS.|K-SPEC\. ED\. OUT OF SCHOOL SUS\.': 'OSS',
                        'G-IN SCHOOL SUSPENSIONS|Q-ECO\. DISADV\. IN SCHOOL SUS\.|L-SPEC\. ED\. IN SCHOOL SUS\.': 'ISS'},
                    "HEADING NAME": {'SPEC\. ED.*$': 'SPE',
                                     'ECO?. DISAD.*$': 'ECO',
                                     'HIS(PANIC)?/LATINO': 'HIS',
                                     'HISPANIC': 'HIS',
                                     '(BLACK)?( OR |/)?AFRICAN AMERICAN': 'BLA',
                                     'WHITE': 'WHI',
                                     'NATIVE AMERICAN':'IND',
                                     'AMERICAN INDIAN OR ALASKA NAT': 'IND',
                                     'ASIAN': 'ASI',
                                     'NATIVE HAWAIIAN/OTHER PACIFIC': 'PCI',
                                     'TWO OR MORE RACES': 'TWO',
                                     'DISTRICT CUMULATIVE YEAR END ENROLLMENT': 'ALL'
                                    }
                    }

    df = apple.replace(to_replace=appleReplace, regex=True)
    
    df["Year"] = year

    for punishment in pdict:
        for category in ("MANDATORY ", "DISCRETIONARY "):
            df.loc[((df['HEADING NAME'] == category + punishment), 'SECTION')] = category + pdict[punishment]
            df = df.replace(to_replace=category + punishment, value="ALL")

    return df


In [3]:
def populations(districtPath, year):
    district = pd.read_csv(districtPath)

    district = district.rename(columns = {"SNAPDIST": 'DISTNAME'})
    
    sometimes_missing = [ 'DPETINDP', 'DPETASIP', 'DPETPCIP', 'DPETTWOP']
    
    for c in sometimes_missing:
        if c not in district.columns:
            district[c] = np.nan
    
    # deleting redundant columns
    
    district["Year"] = year
    
    # dropping 'DPETALLC', which is also a measure of district population, but it's not
    # the same as what the TEA uses in the annual discipline reports processed above.
    
    district = district[['DISTRICT', 'Year', 
                         'DPETBLAP', 'DPETHISP', 'DPETWHIP', 'DPETINDP',
                         'DPETASIP', 'DPETPCIP', 'DPETTWOP', 'DPETECOP', 
                         'DPETSPEP']] 

    district = district.set_index(["DISTRICT",'Year'])
    
    # turning percentages into decimals

    district = district * .01

    return district

In [4]:
def impossible(distPop, racePop, all_punishments, group_punishments):

    """
    >>> print(impossible(50, 20, 20, 100))
    1
    >>> impossible(20, 0, 20, 0)
    0
    """

    # flags implausible data entries. Some of them could still be true if school administrators
    # applied different standards different standards to determine which students belong to which demographic group.
    # Or some could be the result of students not being counted because of the time they moved in and out of district.

    if group_punishments > max(all_punishments,8): # eight because TEA could report 2 masked columns with 4 each
        return 1
    if racePop == 0 and group_punishments > 0:
        return 1
    return 0



def getFisher(row, state_pop, state_punishments, d, p):
    
    # I don't know if this is a valid way to report the Fisher's exact test statistic, but the idea is that if getFisher returns a
    # positive number over .95, there's a 95% chance that the group's better-than-average treatment is not due to chance.
    # If it returns a number under -.95, there's a 95% chance that the group's worse-than-average treatment is not due to chance.
    # I think it should be easier to create a color scale to show the scores on a map this way.

    # The getFisher function assumes wrongly that everyone can have only one punishment (of each type). If the number of
    # punishments exceeds the number of kids, it reduces the number of punishments (and assumes wrongly that every
    # kid has been punished) But maybe the results are still close enough to correct to use for scaling?

    # Using the impossible function to decide whether to proceed, instead of making separate columns.

    if d == "ALL":
        distPop = state_pop # variable names are misleading in the "ALL" case
        all_punishments = state_punishments
        
    else:
        distPop = row[("ALL", "POP")]
        all_punishments = row[("ALL", p)]
    
    racePop = row[(d, "POP")]
    group_punishments = row[(d, p)]
    if pd.isna(group_punishments):
            group_punishments = 0
    if pd.isna(racePop):
        racePop = group_punishments
        
    if impossible(distPop, racePop, all_punishments, group_punishments):
        return None
    
    if max(racePop, group_punishments) == 0 or None:
        return None
    if distPop == 0 or pd.isna(distPop):
        return None
    elif max(group_punishments, all_punishments) == 0 or None:
        return 0
    else:
        if pd.isna(all_punishments):
            all_punishments = 0
        try: 
            oddsratio, pvalueG = stats.fisher_exact([[racePop, max(distPop - racePop, 0)],
                                                 [group_punishments, max(all_punishments - group_punishments, 0)]],
                                                alternative='greater')
        except ValueError:
            print(distPop, racePop, all_punishments, group_punishments)
        oddsratio, pvalueL = stats.fisher_exact([[racePop, max(distPop - racePop, 0)],
                                                 [group_punishments, max(all_punishments - group_punishments, 0)]],
                                                alternative='less')
        if pvalueL < pvalueG:
            pv = 1 - pvalueL
        else:
            pv = pvalueG - 1
        
        # To save space in the output file, this simplifies the decimal values to an integer from -6 to 6
        # It should replace similar code in txappleseedmap/js/index.js
        
        scale = -6
        scale_colors = (-0.99999,-0.9984,-0.992,-0.96,-0.8,-0.2,0.2,0.8,0.96,0.992,0.9984,0.99999)
        
        for v in scale_colors:
            if pv > v:
                scale += 1
        
    return scale

# print(getFisher(20, 5, 20, 10))

In [5]:
def getYear(year):
    year_col = "YR{}".format(str(year)[-2:])
    apple_path = '../data/from_agency/by_region/REGION_{}_DISTRICT_summary_{}.csv'
    one_year = [pd.read_csv(apple_path.format(str(region).zfill(2),str(year)[-2:]), 
                            dtype = {year_col: int})
                for region in range(1,21)]
    a = pd.concat(one_year)
    # a = a[~a.index.duplicated(keep='last')]  # a single row was causing a non-unique multiindex error 
    # dfnames = a[["DISTRICT", "DISTNAME"]].drop_duplicates()
    df = formatDF(a, year, year_col).rename(columns={year_col: "Count"})
    
    dfpivot = df.pivot_table(index=['DISTRICT','Year'], columns=['HEADING NAME', 'SECTION'], values='Count')
    
    # Trying two methods to get total disciplinary actions per district: adding up "Mandatory" and "Discretionary"
    # actions (which are not always reported), and adding up actions against special ed and non-special ed students.
    # Relying on whichever number is higher, on the assumption that if actions are reported anywhere, they probably
    # really happened.

    for p in pdict.values():
        try:
            dfpivot["ALL", p] = dfpivot["ALL"]["DISCRETIONARY " + p] + dfpivot["ALL"]["MANDATORY " + p]
        except KeyError:
            print("No mandatory/discretionary columns for " + p + " in " + str(year))
        dfpivot["ALLS", p] = dfpivot["SPE"][p] + dfpivot["NON SPE"][p]
        dfpivot = dfpivot.drop("DISCRETIONARY " + p, axis=1, level=1)
        dfpivot = dfpivot.drop("MANDATORY " + p, axis=1, level=1)
        try:
            dfpivot["ALL", p] = dfpivot[[("ALL", p), ("ALLS", p)]].max(axis=1)
        except KeyError:
            dfpivot["ALL", p] = dfpivot["ALLS", p]
    
    # Dropping columns that were just used to get totals, and won't be needed for the map.

    dfpivot = dfpivot.drop("ALLS", axis=1, level=0)
    dfpivot = dfpivot.drop("NON SPE", axis=1, level=0)
    dfpivot = dfpivot.sort_index(axis=1) # sorting columns back into order
    
    districtPath = '../data/from_agency/districts/district{}.dat'.format(year)
    district = populations(districtPath, year)

    dfwide = pd.concat([dfpivot, district], axis=1, join='outer')
    
    # adding population for each demographic group
    
    for demo in demos:
        if demo != "ALL":
            dfwide[demo, "POP"] = dfwide[[("ALL", "POP"), ("DPET" + demo + "P")]].prod(axis=1).round(0)
    
    dfpivot = dfwide.drop(columns=["DPETBLAP","DPETHISP","DPETWHIP","DPETINDP","DPETASIP","DPETPCIP","DPETTWOP","DPETECOP","DPETSPEP"])
    dfpivot = dfpivot.sort_index(axis=1)
    
    state_pop = dfpivot[("ALL", "POP")].sum()
    
    for p in pdict.values():
        state_punishments = dfpivot[("ALL", p)].sum()
        for d in demos:
            try:
                dfpivot[(d, p, "S")] = dfpivot.apply(getFisher, axis = 1, args = (state_pop, state_punishments, d, p))
            except KeyError:
                dfpivot[(d, p, "S")] = np.nan # these blank column are needed to make the dataframes merge
                dfpivot[(d, p)] = np.nan
                print("{} {} not found for year {}".format(d, p, year))
            
    dfpivot = dfpivot.sort_index(axis=1).astype(float)
    
    return dfpivot

In [6]:
years = [x for x in range(first_year, last_year + 1)]

df_list = []

for year in years:
    print("starting " + str(year))
    df_list.append(getYear(year))

starting 2006
TWO EXP not found for year 2006
PCI EXP not found for year 2006
TWO DAE not found for year 2006
PCI DAE not found for year 2006
TWO ISS not found for year 2006
PCI ISS not found for year 2006
TWO OSS not found for year 2006
PCI OSS not found for year 2006
starting 2007
TWO EXP not found for year 2007
PCI EXP not found for year 2007
TWO DAE not found for year 2007
PCI DAE not found for year 2007
TWO ISS not found for year 2007
PCI ISS not found for year 2007
TWO OSS not found for year 2007
PCI OSS not found for year 2007
starting 2008
TWO EXP not found for year 2008
PCI EXP not found for year 2008
TWO DAE not found for year 2008
PCI DAE not found for year 2008
TWO ISS not found for year 2008
PCI ISS not found for year 2008
TWO OSS not found for year 2008
PCI OSS not found for year 2008
starting 2009
TWO EXP not found for year 2009
PCI EXP not found for year 2009
TWO DAE not found for year 2009
PCI DAE not found for year 2009
TWO ISS not found for year 2009
PCI ISS not foun

In [13]:
all_years = pd.concat(df_list)

In [14]:
all_years[-10:]

,,"(ALL, DAE)","(ALL, DAE, S)","(ALL, EXP)","(ALL, EXP, S)","(ALL, ISS)","(ALL, ISS, S)","(ALL, OSS)","(ALL, OSS, S)","(ALL, POP)","(ASI, DAE)","(ASI, DAE, S)","(ASI, EXP)","(ASI, EXP, S)","(ASI, ISS)","(ASI, ISS, S)","(ASI, OSS)","(ASI, OSS, S)","(ASI, POP)","(BLA, DAE)","(BLA, DAE, S)","(BLA, EXP)","(BLA, EXP, S)","(BLA, ISS)","(BLA, ISS, S)","(BLA, OSS)","(BLA, OSS, S)","(BLA, POP)","(ECO, DAE)","(ECO, DAE, S)","(ECO, EXP)","(ECO, EXP, S)","(ECO, ISS)","(ECO, ISS, S)","(ECO, OSS)","(ECO, OSS, S)","(ECO, POP)","(HIS, DAE)","(HIS, DAE, S)","(HIS, EXP)","(HIS, EXP, S)","(HIS, ISS)","(HIS, ISS, S)","(HIS, OSS)","(HIS, OSS, S)","(HIS, POP)","(IND, DAE)","(IND, DAE, S)","(IND, EXP)","(IND, EXP, S)","(IND, ISS)","(IND, ISS, S)","(IND, OSS)","(IND, OSS, S)","(IND, POP)","(PCI, DAE)","(PCI, DAE, S)","(PCI, EXP)","(PCI, EXP, S)","(PCI, ISS)","(PCI, ISS, S)","(PCI, OSS)","(PCI, OSS, S)","(PCI, POP)","(SPE, DAE)","(SPE, DAE, S)","(SPE, EXP)","(SPE, EXP, S)","(SPE, ISS)","(SPE, ISS, S)","(SPE, OSS)","(SPE, OSS, S)","(SPE, POP)","(TWO, DAE)","(TWO, DAE, S)","(TWO, EXP)","(TWO, EXP, S)","(TWO, ISS)","(TWO, ISS, S)","(TWO, OSS)","(TWO, OSS, S)","(TWO, POP)","(WHI, DAE)","(WHI, DAE, S)","(WHI, EXP)","(WHI, EXP, S)","(WHI, ISS)","(WHI, ISS, S)","(WHI, OSS)","(WHI, OSS, S)","(WHI, POP)"
DISTRICT,Year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
250906,2016,NaN,-6.0,NaN,-1.0,2.0,-6.0,38.0,-6.0,941.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,5.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,-1.0,6.0,1.0,1.0,NaN,0.0,1.0,-1.0,24.0,2.0,485.0,1.0,2.0,NaN,0.0,1.0,2.0,5.0,1.0,96.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,0.0,1.0,2.0,16.0,6.0,92.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,-1.0,10.0,1.0,0.0,NaN,0.0,1.0,-1.0,33.0,-1.0,821.0
250907,2016,NaN,-6.0,NaN,-1.0,232.0,-6.0,24.0,-6.0,1606.0,NaN,0.0,NaN,0.0,1.0,-1.0,NaN,-1.0,16.0,NaN,0.0,NaN,0.0,1.0,-2.0,1.0,1.0,24.0,1.0,1.0,NaN,0.0,169.0,6.0,18.0,3.0,859.0,NaN,0.0,NaN,0.0,33.0,-1.0,1.0,-2.0,233.0,NaN,0.0,NaN,0.0,NaN,-1.0,NaN,0.0,8.0,NaN,0.0,NaN,0.0,NaN,-1.0,NaN,0.0,2.0,NaN,0.0,NaN,0.0,34.0,5.0,6.0,4.0,117.0,NaN,0.0,NaN,0.0,NaN,-5.0,NaN,-1.0,48.0,5.0,1.0,NaN,0.0,188.0,1.0,16.0,-2.0,1277.0
251901,2016,2.0,-6.0,NaN,-1.0,117.0,-6.0,NaN,-6.0,1853.0,NaN,0.0,NaN,0.0,NaN,-1.0,NaN,0.0,11.0,NaN,0.0,NaN,0.0,1.0,1.0,NaN,0.0,13.0,7.0,3.0,NaN,0.0,58.0,-1.0,1.0,1.0,995.0,12.0,NaN,NaN,0.0,86.0,-2.0,1.0,1.0,1471.0,NaN,0.0,NaN,0.0,1.0,1.0,NaN,0.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,2.0,NaN,0.0,19.0,5.0,NaN,0.0,95.0,NaN,0.0,NaN,0.0,NaN,-1.0,NaN,0.0,13.0,6.0,5.0,NaN,0.0,1.0,-6.0,NaN,0.0,337.0
251902,2016,NaN,-5.0,NaN,-1.0,29.0,-6.0,NaN,-6.0,515.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,2.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,1.0,1.0,1.0,NaN,0.0,20.0,1.0,NaN,0.0,331.0,1.0,1.0,NaN,0.0,20.0,1.0,NaN,0.0,327.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,0.0,5.0,2.0,NaN,0.0,35.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,3.0,NaN,0.0,NaN,0.0,9.0,-1.0,NaN,0.0,182.0
252901,2016,2.0,-6.0,NaN,-1.0,654.0,6.0,45.0,-6.0,2646.0,NaN,0.0,NaN,0.0,5.0,1.0,NaN,0.0,13.0,NaN,0.0,NaN,0.0,6.0,-1.0,1.0,1.0,29.0,1.0,-1.0,NaN,0.0,475.0,6.0,39.0,6.0,1450.0,11.0,NaN,NaN,0.0,228.0,3.0,13.0,-1.0,812.0,NaN,0.0,NaN,0.0,13.0,4.0,1.0,1.0,19.0,NaN,0.0,NaN,0.0,5.0,3.0,NaN,0.0,3.0,1.0,2.0,NaN,0.0,87.0,4.0,11.0,4.0,259.0,NaN,0.0,NaN,0.0,6.0,-1.0,NaN,-1.0,37.0,16.0,NaN,NaN,0.0,391.0,-4.0,26.0,-2.0,1736.0
252902,2016,NaN,-3.0,NaN,0.0,2.0,-6.0,NaN,-6.0,251.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,0.0,16.0,NaN,NaN,0.0,133.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,26.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,0.0,1.0,1.0,NaN,0.0,26.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,13.0,NaN,0.0,NaN,0.0,28.0,NaN,NaN,0.0,212.0
252903,2016,2.0,-5.0,NaN,-1.0,152.0,1.0,2.0,-6.0,738.0,NaN,0.0,NaN,0.0,NaN,-1.0,NaN,0.0,5.0,NaN,0.0,NaN,0.0,1.0,-2.0,1.0,2.0,16.0,1.0,-1.0,NaN,0.0,137.0,6.0,1.0,-1.0,439.0,8.0,5.0,NaN,0.0,78.0,5.0,NaN,-1.0,

In [19]:
# changing column names to something without a comma in them

column_names = {c: '-'.join(c) for c in all_years}
all_years = all_years.rename(column_names, axis='columns')

In [20]:
all_years[:10]

,,ALL-DAE,ALL-DAE-S,ALL-EXP,ALL-EXP-S,ALL-ISS,ALL-ISS-S,ALL-OSS,ALL-OSS-S,ALL-POP,ASI-DAE,ASI-DAE-S,ASI-EXP,ASI-EXP-S,ASI-ISS,ASI-ISS-S,ASI-OSS,ASI-OSS-S,ASI-POP,BLA-DAE,BLA-DAE-S,BLA-EXP,BLA-EXP-S,BLA-ISS,BLA-ISS-S,BLA-OSS,BLA-OSS-S,BLA-POP,ECO-DAE,ECO-DAE-S,ECO-EXP,ECO-EXP-S,ECO-ISS,ECO-ISS-S,ECO-OSS,ECO-OSS-S,ECO-POP,HIS-DAE,HIS-DAE-S,HIS-EXP,HIS-EXP-S,HIS-ISS,HIS-ISS-S,HIS-OSS,HIS-OSS-S,HIS-POP,IND-DAE,IND-DAE-S,IND-EXP,IND-EXP-S,IND-ISS,IND-ISS-S,IND-OSS,IND-OSS-S,IND-POP,PCI-DAE,PCI-DAE-S,PCI-EXP,PCI-EXP-S,PCI-ISS,PCI-ISS-S,PCI-OSS,PCI-OSS-S,PCI-POP,SPE-DAE,SPE-DAE-S,SPE-EXP,SPE-EXP-S,SPE-ISS,SPE-ISS-S,SPE-OSS,SPE-OSS-S,SPE-POP,TWO-DAE,TWO-DAE-S,TWO-EXP,TWO-EXP-S,TWO-ISS,TWO-ISS-S,TWO-OSS,TWO-OSS-S,TWO-POP,WHI-DAE,WHI-DAE-S,WHI-EXP,WHI-EXP-S,WHI-ISS,WHI-ISS-S,WHI-OSS,WHI-OSS-S,WHI-POP
DISTRICT,Year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1902,2006,NaN,-6.0,NaN,-1.0,73.0,-6.0,6.0,-6.0,653.0,NaN,0.0,NaN,0.0,NaN,-6.0,NaN,-6.0,653.0,NaN,0.0,NaN,0.0,1.0,-5.0,NaN,-1.0,91.0,NaN,0.0,NaN,0.0,28.0,-1.0,1.0,-1.0,263.0,NaN,0.0,NaN,0.0,1.0,-1.0,NaN,-1.0,26.0,NaN,0.0,NaN,0.0,NaN,-6.0,NaN,-6.0,653.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,653.0,NaN,0.0,NaN,0.0,28.0,5.0,1.0,1.0,104.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,653.0,1.0,0.0,NaN,0.0,67.0,3.0,6.0,1.0,529.0
1903,2006,19.0,-5.0,NaN,-2.0,479.0,-2.0,13.0,-6.0,1491.0,NaN,-6.0,NaN,0.0,1.0,-6.0,NaN,-6.0,1491.0,1.0,-1.0,NaN,0.0,100.0,6.0,6.0,5.0,134.0,6.0,-2.0,NaN,0.0,253.0,3.0,6.0,-1.0,710.0,1.0,1.0,NaN,0.0,25.0,1.0,NaN,-1.0,75.0,NaN,-6.0,NaN,0.0,NaN,-6.0,NaN,-6.0,1491.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1491.0,8.0,3.0,NaN,0.0,155.0,6.0,5.0,2.0,253.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1491.0,14.0,-2.0,NaN,0.0,352.0,-6.0,7.0,-4.0,1282.0
1904,2006,14.0,-3.0,NaN,-2.0,330.0,1.0,18.0,-6.0,915.0,NaN,-6.0,NaN,0.0,NaN,-6.0,NaN,-6.0,915.0,1.0,-1.0,1.0,2.0,94.0,6.0,7.0,3.0,128.0,9.0,1.0,1.0,1.0,212.0,6.0,14.0,3.0,459.0,NaN,-1.0,NaN,0.0,16.0,-2.0,1.0,-1.0,64.0,NaN,-6.0,NaN,0.0,NaN,-6.0,NaN,-6.0,915.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,915.0,10.0,6.0,1.0,2.0,135.0,6.0,7.0,3.0,128.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,915.0,10.0,-1.0,NaN,0.0,220.0,-6.0,10.0,-3.0,723.0
1906,2006,6.0,-2.0,NaN,-1.0,68.0,-6.0,2.0,-6.0,369.0,NaN,-6.0,NaN,0.0,NaN,-6.0,NaN,-5.0,369.0,1.0,1.0,NaN,0.0,5.0,-1.0,NaN,0.0,30.0,6.0,4.0,NaN,0.0,48.0,6.0,1.0,1.0,140.0,NaN,-1.0,NaN,0.0,1.0,-1.0,NaN,0.0,15.0,NaN,-6.0,NaN,0.0,NaN,-6.0,NaN,-5.0,369.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,369.0,5.0,4.0,NaN,0.0,27.0,5.0,1.0,1.0,77.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,369.0,8.0,1.0,NaN,0.0,59.0,-1.0,1.0,-1.0,321.0
1907,2006,94.0,-2.0,NaN,-5.0,1581.0,5.0,214.0,-6.0,4103.0,1.0,-6.0,NaN,0.0,1.0,-6.0,1.0,-6.0,4103.0,42.0,4.0,NaN,0.0,794.0,6.0,128.0,6.0,1231.0,70.0,3.0,NaN,0.0,1179.0,6.0,159.0,5.0,2605.0,21.0,-2.0,NaN,0.0,389.0,-5.0,28.0,-6.0,1231.0,NaN,-6.0,NaN,0.0,NaN,-6.0,1.0,-6.0,4103.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4103.0,28.0,6.0,NaN,0.0,400.0,6.0,53.0,6.0,451.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4103.0,30.0,-2.0,NaN,0.0,396.0,-6.0,56.0,-5.0,1641.0
1908,2006,52.0,-1.0,2.0,-1.0,695.0,-1.0,97.0,-6.0,2042.0,NaN,-6.0,NaN,-6.0,1.0,-6.0,NaN,-6.0,2042.0,8.0,-1.0,NaN,-1.0,172.0,6.0,40.0,6.0,347.0,15.0,-3.0,1.0,1.0,333.0,3.0,44.0,1.0,892.0,6.0,-1.0,1.0,1.0,76.0,-1.0,18.0,2.0,245.0,NaN,-6.0,NaN,-6.0,1.0,-6.0,NaN,-6.0,2042.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2042.0,8.0,1.0,1.0,1.0,234.0,6.0,24.0,4.0,286.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2042.0,38.0,1.0,1.0,-1.0,445.0,-5.0,39.0,-6.0,1450.0
1909,2006,2.0,-5.0,NaN,-1.0,109.0,-5.0,19.0,-6.0,457.0,NaN,-6.0,NaN,0.0,NaN,-6.0,NaN,-6.0,457.0,NaN,0.0,NaN,0.0,NaN,-2.0,NaN,-1.0,9.0,1.0,-1.0,1.0,1.0,80.0,4.0,14.0,2.0,266.0,1.0,2.0,NaN,0.0,1.0,-3.0,1.0,1.0,23.0,NaN,-6.0,NaN,0.0,NaN,-6.0,NaN,-6.0,457.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,457.0,1.0,1.0,1.0,1.0,53.0,6.0,12.0,5.0,101.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,457.0,1.0,-2.0,1.0,0.0,106.0,2.0,17.0,-1.0,425.0
2901,2006,94.0,1.0,NaN,-4.0,447.0,-6.0,127.0,-6.0,3157.0,NaN,-6.0,NaN,0.0,NaN,-6.0,NaN,-6.0,3157.0,5.0,2.0,NaN,0.0,12.0,1.0,1.0,-1.0,63.0,45.0,

In [21]:
all_years.to_csv("../data/processed/schoolToPrison_flat.csv")

In [ ]:
# Checking to see how to restore the column MultiIndex
# but the output CSV just has a regular index for the columns (dtype: string)

m = pd.MultiIndex.from_tuples(dfpivot.columns)

In [ ]:
df = dfwide.set_axis(m, axis = 1, inplace=False)

In [ ]:
dfpivot = dfpivot.reset_index()

In [ ]:
dfpivot = dfpivot.sort_index(axis=1)

In [ ]:
dfpivot[:5]

In [ ]:
# This function's unused now.

def getRatio(distPop, racePop, all_punishments, group_punishments):
    # Calculating ratio of punishments for the demographic group compared to the punishments for the student population
    # as a whole. For instance, "0.505" in the disparity column indicates the group got the punishment 50.5% as often
    # as average for the student population.

    """
    >>> getRatio(200, 20, 20, 10)
    4.0
    >>> getRatio(200, 20, 20, 2)
    0.0
    >>> print(getRatio(200, 0, 20, 0))
    None
    """

    if max(racePop, group_punishments) == 0 or None:
        return None
    elif all_punishments == 0 or None:
        return 0
    else:
        disparity = (group_punishments / (max(all_punishments, group_punishments))
                     / (max(racePop, group_punishments) / distPop)) - 1
        disparity = Decimal(disparity)
        disparity = disparity.quantize(Decimal('0.01'))
    return float(disparity)